In [1]:
import pandas as pd
import numpy as np
import os
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime, timedelta, date
import operator

from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import KFold
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data_path = 'C:/Users/Administrator/Documents/'
path = 'C:/Users/Administrator/Documents/05_19acb/'
ver = '05_19acb_catboost'

In [3]:
ID = ['air_store_id','visit_date','flag']
TARGET='visitors'

In [4]:
def RMSLE(y, pred):
    return mean_squared_error(y, pred) ** 0.5

In [5]:
train = pd.read_csv(os.path.join(data_path, 'train2.csv'))

In [6]:
# Variable Selection
cols1 = ['holiday_flg','holiday_flg+1d','holiday_flg-1d','holiday_flg_rev',\
        'dow',\
#          'month',\
#             'dow_0',
        'dow_1','dow_2','dow_3','dow_4','dow_5','dow_6',\
#             'month_1',
        'month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12',\
        'air_genre',\
        'latitude','longitude',\
#             'var_max_lat','var_max_long',\
#             'km_latlong',\
        'air_areaL1_lbl',\
        'air_areaL2_lbl',\
#         'air_areaL3_lbl',\
#             'km_latlong_0',\
        'km_latlong_1','km_latlong_2','km_latlong_3','km_latlong_4','km_latlong_5',\
        'km_latlong_6','km_latlong_7','km_latlong_8','km_latlong_9','km_latlong_10','km_latlong_11',\
#             'air_genre_0','air_genre_1','air_genre_2','air_genre_3','air_genre_4','air_genre_5','air_genre_6','air_genre_7',\
#             'air_genre_8','air_genre_9','air_genre_10','air_genre_11','air_genre_12','air_genre_13',\
#             'km_hpg_latlong',\
        'days_from_first_date',\
        'dow_all_wmean','dow_all_max','dow_all_min','dow_all_med',\
#         'dow_all_cnt',\
        'dowhol_all_wmean','dowhol_all_max','dowhol_all_min','dowhol_all_med',\
        'dowhol+1d_all_wmean','dowhol+1d_all_max','dowhol+1d_all_min','dowhol+1d_all_med',\
        'dowhol-1d_all_wmean','dowhol-1d_all_max','dowhol-1d_all_min','dowhol-1d_all_med',\
#             'dowhol_rev_all_wmean','dowhol_rev_all_max','dowhol_rev_all_min','dowhol_rev_all_med','dowhol_rev_all_cnt',\
#             'dowhol_all_med',\
#             'dowhol+1d_all_med',\
#             'dowhol-1d_all_med',\
#         'dowhol_all_cnt',\
#             'dowhol+1d_all_cnt',\
#             'dowhol-1d_all_cnt',\
#             'km_latlong_mean','km_latlong_dow_mean',\
        'genre_mean',\
#          'genre_dow_mean',\
#         'areaL1_mean',\
#         'areaL1_dow_mean',\
#         'areaL2_mean',\
#         'areaL2_dow_mean',\
        'areaL3_mean',\
#         'areaL3_dow_mean',\
        'res_ttl','res_cnt','res_mean',\
        'res_hr_std',\
#         'res_std',\
#         'res_hr_dif_sum',\
#         'res_hr_dif_mean',\
        'res_ttl_dow_mean','res_cnt_dow_mean',\
        'lag_1d','lag_2d','lag_3d','lag_4d','lag_5d','lag_6d',\
        'lag_10d','lag_15d','lag_20d','lag_25d','lag_30d','lag_35d','lag_40d','lag_45d','lag_50d',\
         
        'lag_res_1d','lag_res_2d','lag_res_3d','lag_res_4d','lag_res_5d','lag_res_6d',\
#         'lag_res_10d','lag_res_15d','lag_res_20d',\
#         'lag_res_25d','lag_res_30d','lag_res_35d','lag_res_40d','lag_res_45d','lag_res_50d',\
        'lag_1w',\
        'lag_2w',\
        'lag_3w',\
        'lag_4w',\
        'lag_5w',\
        'lag_6w',\
        'lag_7w',\
        'lag_8w',\
        'lag_9w',\
        'lag_10w',\
        'lag_11w',\
        'lag_12w',\
        'lag_13w',\
        'lag_14w',\
        'lag_15w',\
        'lag_16w',\
        'lag_17w',\
        'lag_18w',\
        'lag_19w',\
        'lag_20w',\
        'lag_res_1w','lag_res_2w','lag_res_3w','lag_res_4w','lag_res_5w',\
        'lag_res_6w','lag_res_7w','lag_res_8w','lag_res_9w','lag_res_10w',\
#         'lag_res_11w','lag_res_12w','lag_res_13w','lag_res_14w','lag_res_15w',\
#         'lag_res_16w','lag_res_17w','lag_res_18w','lag_res_19w','lag_res_20w',\
        'mean_3d','max_3d','min_3d','std_3d',\
        'mean_7d','max_7d','min_7d','std_7d',\
        'mean_14d','max_14d','min_14d','std_14d',\
#         'mean_21d','max_21d','min_21d','std_21d',\
        'mean_28d','max_28d','min_28d','std_28d',\
#         'mean_35d','max_35d','min_35d','std_35d',\
        'mean_42d','max_42d','min_42d','std_42d',\
#             'mean_49d','max_49d','min_49d','std_49d',\
        'mean_56d','max_56d','min_56d','std_56d',\
#             'mean_63d','max_63d','min_63d','std_63d',\
#         'mean_70d','max_70d','min_70d','std_70d',\
#             'mean_77d','max_77d','min_77d','std_77d',\
        'mean_84d','max_84d','min_84d','std_84d',\
#             'scale_to_maxmin_3d','scale_to_std_3d',\
        'scale_to_maxmin_7d','scale_to_std_7d',\
        'scale_to_maxmin_14d','scale_to_std_14d',\
#             'scale_to_maxmin_21d','scale_to_std_21d',\
        'scale_to_maxmin_28d','scale_to_std_28d',\
#             'scale_to_maxmin_35d','scale_to_std_35d',\
#             'scale_to_maxmin_42d','scale_to_std_42d',\
#             'scale_to_maxmin_49d','scale_to_std_49d',\
        'scale_to_maxmin_56d','scale_to_std_56d',\
#             'scale_to_maxmin_63d','scale_to_std_63d',\
#             'scale_to_maxmin_70d','scale_to_std_70d',\
#             'scale_to_maxmin_77d','scale_to_std_77d',\
        'scale_to_maxmin_84d','scale_to_std_84d',\
#             'mean_7d-14d','mean_7d-28d','mean_7d-56d','mean_7d-84d','mean_14d-28d','mean_14d-56d','mean_14d-84d',\
#             'mean_28d-56d','mean_28d-84d','mean_56d-84d',\
        'mean_dow_2w','max_dow_2w','min_dow_2w','std_dow_2w',\
#         'mean_dow_3w','max_dow_3w','min_dow_3w','std_dow_3w',\
        'mean_dow_4w','max_dow_4w','min_dow_4w','std_dow_4w',\
#             'mean_dow_5w','max_dow_5w','min_dow_5w','std_dow_5w',\
        'mean_dow_6w','max_dow_6w','min_dow_6w','std_dow_6w',\
#             'mean_dow_7w','max_dow_7w','min_dow_7w','std_dow_7w',\
#         'mean_dow_8w','max_dow_8w','min_dow_8w','std_dow_8w',\
#             'mean_dow_9w','max_dow_9w','min_dow_9w','std_dow_9w',\
#             'mean_dow_10w','max_dow_10w','min_dow_10w','std_dow_10w',\
#             'mean_dow_2w-4w','mean_dow_2w-6w','mean_dow_2w-8w','mean_dow_2w-10w',\
#         'mean_dow_4w-6w','mean_dow_4w-8w','mean_dow_4w-10w',\
#         'mean_dow_6w-8w','mean_dow_6w-10w','mean_dow_8w-10w',\
        'rainfall','snowfall','wind_max',\
        'weather_daytime',\
        'weather_nighttime',\
#             'weather_daytime2','weather_nighttime2',\
        'weather_daytime_0',\
        'weather_daytime_1',\
#         'weather_daytime_2',\
        'weather_daytime_3',\
        'weather_nighttime_0',\
        'weather_nighttime_1',\
        'weather_nighttime_2',\
        'weather_nighttime_3',\
#         'wind_max_inst',\
#         'rainfall_max1h',\
#         'wind_avg',\
        'temperature_high','temperature_low','temperature_avg',\
#         'snowfall_max',\
#         'humidity_avg',\
#         'daylight_hr',\
#         'na_cnt'
        ]
# add = [c for c in train.columns if "air_areaL1_lbl_" in c]
# cols.extend(add)
# add = [c for c in train.columns if "air_areaL2_lbl_" in c]
# cols.extend(add)
add = [c for c in train.columns if "air_areaL3_lbl_" in c]
cols1.extend(add)

In [7]:
# Variable Selection
cols2 = ['holiday_flg','holiday_flg+1d','holiday_flg-1d','holiday_flg_rev',\
        'dow',\
#          'month',\
#             'dow_0',
        'dow_1','dow_2','dow_3','dow_4','dow_5','dow_6',\
#             'month_1',
        'month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12',\
        'air_genre',\
        'latitude','longitude',\
#             'var_max_lat','var_max_long',\
#             'km_latlong',\
        'air_areaL1_lbl',\
        'air_areaL2_lbl',\
#         'air_areaL3_lbl',\
#             'km_latlong_0',\
        'km_latlong_1','km_latlong_2','km_latlong_3','km_latlong_4','km_latlong_5',\
        'km_latlong_6','km_latlong_7','km_latlong_8','km_latlong_9','km_latlong_10','km_latlong_11',\
#             'air_genre_0','air_genre_1','air_genre_2','air_genre_3','air_genre_4','air_genre_5','air_genre_6','air_genre_7',\
#             'air_genre_8','air_genre_9','air_genre_10','air_genre_11','air_genre_12','air_genre_13',\
#             'km_hpg_latlong',\
        'days_from_first_date',\
        'dow_all_wmean','dow_all_max','dow_all_min','dow_all_med',\
#         'dow_all_cnt',\
        'dowhol_all_wmean','dowhol_all_max','dowhol_all_min','dowhol_all_med',\
        'dowhol+1d_all_wmean','dowhol+1d_all_max','dowhol+1d_all_min','dowhol+1d_all_med',\
        'dowhol-1d_all_wmean','dowhol-1d_all_max','dowhol-1d_all_min','dowhol-1d_all_med',\
#             'dowhol_rev_all_wmean','dowhol_rev_all_max','dowhol_rev_all_min','dowhol_rev_all_med','dowhol_rev_all_cnt',\
#             'dowhol_all_med',\
#             'dowhol+1d_all_med',\
#             'dowhol-1d_all_med',\
#         'dowhol_all_cnt',\
#             'dowhol+1d_all_cnt',\
#             'dowhol-1d_all_cnt',\
#             'km_latlong_mean','km_latlong_dow_mean',\
        'genre_mean',\
#          'genre_dow_mean',\
#         'areaL1_mean',\
#         'areaL1_dow_mean',\
#         'areaL2_mean',\
#         'areaL2_dow_mean',\
        'areaL3_mean',\
#         'areaL3_dow_mean',\
        'res_ttl','res_cnt','res_mean',\
        'res_hr_std',\
#         'res_std',\
#         'res_hr_dif_sum',\
#         'res_hr_dif_mean',\
        'res_ttl_dow_mean','res_cnt_dow_mean',\
#         'lag_1d','lag_2d','lag_3d','lag_4d','lag_5d','lag_6d',\
        'lag_10d','lag_15d','lag_20d','lag_25d','lag_30d','lag_35d','lag_40d','lag_45d','lag_50d',\
         
        'lag_res_1d','lag_res_2d','lag_res_3d','lag_res_4d','lag_res_5d','lag_res_6d',\
#         'lag_res_10d','lag_res_15d','lag_res_20d',\
#         'lag_res_25d','lag_res_30d','lag_res_35d','lag_res_40d','lag_res_45d','lag_res_50d',\
        'lag_1w',\
        'lag_2w',\
        'lag_3w',\
        'lag_4w',\
        'lag_5w',\
        'lag_6w',\
        'lag_7w',\
        'lag_8w',\
        'lag_9w',\
        'lag_10w',\
        'lag_11w',\
        'lag_12w',\
        'lag_13w',\
        'lag_14w',\
        'lag_15w',\
        'lag_16w',\
        'lag_17w',\
        'lag_18w',\
        'lag_19w',\
        'lag_20w',\
        'lag_res_1w','lag_res_2w','lag_res_3w','lag_res_4w','lag_res_5w',\
        'lag_res_6w','lag_res_7w','lag_res_8w','lag_res_9w','lag_res_10w',\
#         'lag_res_11w','lag_res_12w','lag_res_13w','lag_res_14w','lag_res_15w',\
#         'lag_res_16w','lag_res_17w','lag_res_18w','lag_res_19w','lag_res_20w',\
        'mean_3d','max_3d','min_3d','std_3d',\
        'mean_7d','max_7d','min_7d','std_7d',\
        'mean_14d','max_14d','min_14d','std_14d',\
#         'mean_21d','max_21d','min_21d','std_21d',\
        'mean_28d','max_28d','min_28d','std_28d',\
#         'mean_35d','max_35d','min_35d','std_35d',\
        'mean_42d','max_42d','min_42d','std_42d',\
#             'mean_49d','max_49d','min_49d','std_49d',\
        'mean_56d','max_56d','min_56d','std_56d',\
#             'mean_63d','max_63d','min_63d','std_63d',\
#         'mean_70d','max_70d','min_70d','std_70d',\
#             'mean_77d','max_77d','min_77d','std_77d',\
        'mean_84d','max_84d','min_84d','std_84d',\
#             'scale_to_maxmin_3d','scale_to_std_3d',\
        'scale_to_maxmin_7d','scale_to_std_7d',\
        'scale_to_maxmin_14d','scale_to_std_14d',\
#             'scale_to_maxmin_21d','scale_to_std_21d',\
        'scale_to_maxmin_28d','scale_to_std_28d',\
#             'scale_to_maxmin_35d','scale_to_std_35d',\
#             'scale_to_maxmin_42d','scale_to_std_42d',\
#             'scale_to_maxmin_49d','scale_to_std_49d',\
        'scale_to_maxmin_56d','scale_to_std_56d',\
#             'scale_to_maxmin_63d','scale_to_std_63d',\
#             'scale_to_maxmin_70d','scale_to_std_70d',\
#             'scale_to_maxmin_77d','scale_to_std_77d',\
        'scale_to_maxmin_84d','scale_to_std_84d',\
#             'mean_7d-14d','mean_7d-28d','mean_7d-56d','mean_7d-84d','mean_14d-28d','mean_14d-56d','mean_14d-84d',\
#             'mean_28d-56d','mean_28d-84d','mean_56d-84d',\
        'mean_dow_2w','max_dow_2w','min_dow_2w','std_dow_2w',\
#         'mean_dow_3w','max_dow_3w','min_dow_3w','std_dow_3w',\
        'mean_dow_4w','max_dow_4w','min_dow_4w','std_dow_4w',\
#             'mean_dow_5w','max_dow_5w','min_dow_5w','std_dow_5w',\
        'mean_dow_6w','max_dow_6w','min_dow_6w','std_dow_6w',\
#             'mean_dow_7w','max_dow_7w','min_dow_7w','std_dow_7w',\
#         'mean_dow_8w','max_dow_8w','min_dow_8w','std_dow_8w',\
#             'mean_dow_9w','max_dow_9w','min_dow_9w','std_dow_9w',\
#             'mean_dow_10w','max_dow_10w','min_dow_10w','std_dow_10w',\
#             'mean_dow_2w-4w','mean_dow_2w-6w','mean_dow_2w-8w','mean_dow_2w-10w',\
#         'mean_dow_4w-6w','mean_dow_4w-8w','mean_dow_4w-10w',\
#         'mean_dow_6w-8w','mean_dow_6w-10w','mean_dow_8w-10w',\
        'rainfall','snowfall','wind_max',\
        'weather_daytime',\
        'weather_nighttime',\
#             'weather_daytime2','weather_nighttime2',\
        'weather_daytime_0',\
        'weather_daytime_1',\
#         'weather_daytime_2',\
        'weather_daytime_3',\
        'weather_nighttime_0',\
        'weather_nighttime_1',\
        'weather_nighttime_2',\
        'weather_nighttime_3',\
#         'wind_max_inst',\
#         'rainfall_max1h',\
#         'wind_avg',\
        'temperature_high','temperature_low','temperature_avg',\
#         'snowfall_max',\
#         'humidity_avg',\
#         'daylight_hr',\
#         'na_cnt'
        ]
# add = [c for c in train.columns if "air_areaL1_lbl_" in c]
# cols.extend(add)
# add = [c for c in train.columns if "air_areaL2_lbl_" in c]
# cols.extend(add)
add = [c for c in train.columns if "air_areaL3_lbl_" in c]
cols2.extend(add)

In [8]:
cols_adj = ['dow_all_wmean','dow_all_max','dow_all_min','dow_all_med',\
            'dowhol_all_wmean','dowhol_all_max','dowhol_all_min','dowhol_all_med',\
            'dowhol+1d_all_wmean','dowhol+1d_all_max','dowhol+1d_all_min','dowhol+1d_all_med',\
            'dowhol-1d_all_wmean','dowhol-1d_all_max','dowhol-1d_all_min','dowhol-1d_all_med',\
            'dowhol_rev_all_wmean','dowhol_rev_all_max','dowhol_rev_all_min','dowhol_rev_all_med',\
            'km_latlong_mean','km_latlong_dow_mean',\
            'genre_mean','genre_dow_mean',\
            'areaL1_mean','areaL1_dow_mean',\
            'areaL2_mean','areaL2_dow_mean',\
            'areaL3_mean','areaL3_dow_mean',\
            'mean_3d','max_3d','min_3d','std_3d','scale_to_maxmin_3d','scale_to_std_3d',\
            'mean_7d','max_7d','min_7d','std_7d','scale_to_maxmin_7d','scale_to_std_7d',\
            'mean_14d','max_14d','min_14d','std_14d','scale_to_maxmin_14d','scale_to_std_14d',\
            'mean_21d','max_21d','min_21d','std_21d','scale_to_maxmin_21d','scale_to_std_21d',\
            'mean_28d','max_28d','min_28d','std_28d','scale_to_maxmin_28d','scale_to_std_28d',\
            'mean_35d','max_35d','min_35d','std_35d','scale_to_maxmin_35d','scale_to_std_35d',\
            'mean_42d','max_42d','min_42d','std_42d','scale_to_maxmin_42d','scale_to_std_42d',\
            'mean_49d','max_49d','min_49d','std_49d','scale_to_maxmin_49d','scale_to_std_49d',\
            'mean_56d','max_56d','min_56d','std_56d','scale_to_maxmin_56d','scale_to_std_56d',\
            'mean_63d','max_63d','min_63d','std_63d','scale_to_maxmin_63d','scale_to_std_63d',\
            'mean_70d','max_70d','min_70d','std_70d','scale_to_maxmin_70d','scale_to_std_70d',\
            'mean_77d','max_77d','min_77d','std_77d','scale_to_maxmin_77d','scale_to_std_77d',\
            'mean_84d','max_84d','min_84d','std_84d','scale_to_maxmin_84d','scale_to_std_84d',\
            'mean_7d-14d','mean_7d-28d','mean_7d-56d','mean_7d-84d','mean_14d-28d','mean_14d-56d','mean_14d-84d',\
            'mean_28d-56d','mean_28d-84d','mean_56d-84d']
cols_remove = ['dow_all_cnt','dowhol_all_cnt','dowhol+1d_all_cnt','dowhol-1d_all_cnt','dowhol_rev_all_cnt',
               'lag_1d','lag_2d','lag_3d','lag_4d','lag_5d','lag_6d']
cols_inf = ['scale_to_maxmin_3d','scale_to_std_3d',\
            'scale_to_maxmin_7d','scale_to_std_7d',\
            'scale_to_maxmin_14d','scale_to_std_14d',\
            'scale_to_maxmin_21d','scale_to_std_21d',\
            'scale_to_maxmin_28d','scale_to_std_28d',\
            'scale_to_maxmin_35d','scale_to_std_35d',\
            'scale_to_maxmin_42d','scale_to_std_42d',\
            'scale_to_maxmin_49d','scale_to_std_49d',\
            'scale_to_maxmin_56d','scale_to_std_56d',\
            'scale_to_maxmin_63d','scale_to_std_63d',\
            'scale_to_maxmin_70d','scale_to_std_70d',\
            'scale_to_maxmin_77d','scale_to_std_77d',\
            'scale_to_maxmin_84d','scale_to_std_84d']

In [ ]:
def model_run(df):
    train = pd.read_csv(os.path.join(data_path, df+'.csv'))
    train['visit_date'] = pd.to_datetime(train['visit_date'])
    time_index = np.where(train.visit_date>=(date(2017,4,23)-timedelta(days=365)))
    train = train.loc[time_index].reset_index()
    
    test = pd.read_csv(os.path.join(data_path, 'test_all.csv'))
    if df=='train1':
        test = test.loc[test.flag==1,:].reset_index(drop=True)
    elif df=='train2':
        test = test.loc[test.flag==2,:].reset_index(drop=True)
        
    y_train = np.log1p(train['visitors'])
    id_train = train[ID]
    id_test  = test[ID]
    x_train = train
    x_test = test
    x_train.drop(ID, axis=1, inplace=True)
    x_train.drop(TARGET, axis=1, inplace=True)
    x_test.drop(ID, axis=1, inplace=True)
    del train, test
    
    tr_te = pd.concat([x_train,x_test])
    for x in cols_inf:
        tr_te[x] = tr_te[x].replace(-np.inf,tr_te[x].median())
        tr_te[x] = tr_te[x].replace(np.inf,tr_te[x].median())
    for x in cols_adj:
        tr_te[x] = tr_te[x].replace(-1,tr_te[x].median())
        
    ntrain = x_train.shape[0]
    x_train = tr_te[:ntrain]
    x_test  = tr_te[ntrain:]

    if df=='train1':
        x_train = x_train[cols1]
        x_test = x_test[cols1]
    elif df=='train2':
        x_train = x_train[cols2]
        x_test = x_test[cols2]
        
#     x_train['areaL3_mean_x_genre_mean'] = x_train['areaL3_mean']*x_train['genre_mean'] 
#     x_test['areaL3_mean_x_genre_mean']  = x_test['areaL3_mean']*x_test['genre_mean'] 
    
    x_train['wind_max_x_rainfall'] = x_train['wind_max']*x_train['rainfall'] 
    x_test['wind_max_x_rainfall']  = x_test['wind_max']*x_test['rainfall']

    n_folds = 5
    cv_sum = 0
    pred_tr = []
    pred_te = []

    kf = KFold(x_train.shape[0], n_folds=n_folds, random_state=1234, shuffle=True)

    model = CatBoostRegressor(
            #eval_metric=['MAE'],
            random_seed=0,
            iterations=5000,
            learning_rate=0.05,
            depth=9,
            use_best_model=True,
            verbose=True)

    for i, (tr_index,vl_index) in enumerate(kf):
        print('\n Fold %d' % (i+1))
        pred = []
        pred_vl = []

        x_tr, x_vl = x_train.loc[tr_index].copy(), x_train.loc[vl_index].copy()
        y_tr, y_vl = y_train[tr_index], y_train[vl_index]
        id_vl      = id_train.loc[vl_index].copy()

        clf = model.fit(X=x_tr,
                        y=y_tr,
                        eval_set=(x_vl, y_vl),
                        plot=True)

        pred = clf.predict(x_vl)

        cv_score = RMSLE(y_vl, pred)
        print('RMSLE: %.6f' % cv_score)

        pred_vl = id_vl
        pred_vl['pred'] = pred    
        if i==0:
            pred_tr = pred_vl
        else:
            pred_tr = pd.concat([pred_tr,pred_vl])

        pred = clf.predict(x_test)
        if i==0:
            pred_te = pred
        else:
            pred_te += pred
        cv_sum = cv_sum + cv_score

    pred_te /= n_folds
    pred_te = pd.DataFrame({'pred':pred_te})
    pred_te = pd.concat([id_test,pred_te],axis=1)

    print('CV RMSLE: %.6f' % (cv_sum / n_folds))

    return pred_tr, pred_te

In [ ]:
pred_tr1, pred_te1 = model_run('train1')
pred_tr1.to_csv(path+'/'+ver+'_pred_tr1.csv', index=False)
pred_te1.to_csv(path+'/'+ver+'_pred_te1.csv', index=False)


 Fold 1


A Jupyter Widget

0:	learn: 2.7706640	test: 2.7752028	best: 2.7752028 (0)	total: 590ms	remaining: 49m 7s
1:	learn: 2.6376553	test: 2.6422302	best: 2.6422302 (1)	total: 1.08s	remaining: 44m 52s
2:	learn: 2.5116005	test: 2.5162272	best: 2.5162272 (2)	total: 1.56s	remaining: 43m 24s
3:	learn: 2.3920516	test: 2.3967478	best: 2.3967478 (3)	total: 2.05s	remaining: 42m 40s
4:	learn: 2.2787372	test: 2.2835263	best: 2.2835263 (4)	total: 2.53s	remaining: 42m 6s
5:	learn: 2.1713875	test: 2.1762757	best: 2.1762757 (5)	total: 2.97s	remaining: 41m 13s
6:	learn: 2.0696915	test: 2.0746826	best: 2.0746826 (6)	total: 3.41s	remaining: 40m 33s
7:	learn: 1.9733665	test: 1.9784000	best: 1.9784000 (7)	total: 3.89s	remaining: 40m 29s
8:	learn: 1.8821716	test: 1.8872887	best: 1.8872887 (8)	total: 4.34s	remaining: 40m 4s
9:	learn: 1.7959007	test: 1.8011003	best: 1.8011003 (9)	total: 4.78s	remaining: 39m 47s
10:	learn: 1.7142116	test: 1.7194794	best: 1.7194794 (10)	total: 5.26s	remaining: 39m 44s
11:	learn: 1.6370303	test: 1.6423

92:	learn: 0.5075407	test: 0.5157061	best: 0.5157061 (92)	total: 44s	remaining: 38m 43s
93:	learn: 0.5073708	test: 0.5155602	best: 0.5155602 (93)	total: 44.5s	remaining: 38m 44s
94:	learn: 0.5071771	test: 0.5154262	best: 0.5154262 (94)	total: 45s	remaining: 38m 42s
95:	learn: 0.5070293	test: 0.5153056	best: 0.5153056 (95)	total: 45.4s	remaining: 38m 40s
96:	learn: 0.5067868	test: 0.5151357	best: 0.5151357 (96)	total: 45.9s	remaining: 38m 41s
97:	learn: 0.5066260	test: 0.5150103	best: 0.5150103 (97)	total: 46.4s	remaining: 38m 39s
98:	learn: 0.5065166	test: 0.5149080	best: 0.5149080 (98)	total: 46.9s	remaining: 38m 39s
99:	learn: 0.5063570	test: 0.5147909	best: 0.5147909 (99)	total: 47.3s	remaining: 38m 39s
100:	learn: 0.5062159	test: 0.5146888	best: 0.5146888 (100)	total: 47.8s	remaining: 38m 37s
101:	learn: 0.5060556	test: 0.5145809	best: 0.5145809 (101)	total: 48.3s	remaining: 38m 37s
102:	learn: 0.5058941	test: 0.5144502	best: 0.5144502 (102)	total: 48.8s	remaining: 38m 37s
103:	lea

182:	learn: 0.4980159	test: 0.5103071	best: 0.5103071 (182)	total: 1m 26s	remaining: 37m 52s
183:	learn: 0.4979548	test: 0.5102998	best: 0.5102998 (183)	total: 1m 26s	remaining: 37m 52s
184:	learn: 0.4979152	test: 0.5102801	best: 0.5102801 (184)	total: 1m 27s	remaining: 37m 52s
185:	learn: 0.4978592	test: 0.5102612	best: 0.5102612 (185)	total: 1m 27s	remaining: 37m 52s
186:	learn: 0.4977917	test: 0.5102321	best: 0.5102321 (186)	total: 1m 28s	remaining: 37m 50s
187:	learn: 0.4977248	test: 0.5102077	best: 0.5102077 (187)	total: 1m 28s	remaining: 37m 50s
188:	learn: 0.4976651	test: 0.5101928	best: 0.5101928 (188)	total: 1m 29s	remaining: 37m 50s
189:	learn: 0.4975592	test: 0.5101365	best: 0.5101365 (189)	total: 1m 29s	remaining: 37m 50s
190:	learn: 0.4974878	test: 0.5101211	best: 0.5101211 (190)	total: 1m 30s	remaining: 37m 49s
191:	learn: 0.4974298	test: 0.5100966	best: 0.5100966 (191)	total: 1m 30s	remaining: 37m 49s
192:	learn: 0.4973675	test: 0.5100639	best: 0.5100639 (192)	total: 1m 

...

4784:	learn: 0.3439895	test: 0.5029034	best: 0.5021562 (2715)	total: 38m 26s	remaining: 1m 43s
4785:	learn: 0.3439711	test: 0.5029058	best: 0.5021562 (2715)	total: 38m 26s	remaining: 1m 43s
4786:	learn: 0.3439457	test: 0.5029115	best: 0.5021562 (2715)	total: 38m 27s	remaining: 1m 42s
4787:	learn: 0.3439220	test: 0.5029104	best: 0.5021562 (2715)	total: 38m 27s	remaining: 1m 42s
4788:	learn: 0.3438900	test: 0.5029120	best: 0.5021562 (2715)	total: 38m 28s	remaining: 1m 41s
4789:	learn: 0.3438672	test: 0.5029091	best: 0.5021562 (2715)	total: 38m 28s	remaining: 1m 41s
4790:	learn: 0.3438344	test: 0.5029099	best: 0.5021562 (2715)	total: 38m 29s	remaining: 1m 40s
4791:	learn: 0.3438068	test: 0.5029214	best: 0.5021562 (2715)	total: 38m 29s	remaining: 1m 40s
4792:	learn: 0.3437907	test: 0.5029220	best: 0.5021562 (2715)	total: 38m 30s	remaining: 1m 39s
4793:	learn: 0.3437602	test: 0.5029214	best: 0.5021562 (2715)	total: 38m 30s	remaining: 1m 39s
4794:	learn: 0.3437324	test: 0.5029229	best: 0.502

4871:	learn: 0.3417236	test: 0.5030052	best: 0.5021562 (2715)	total: 39m 9s	remaining: 1m 1s
4872:	learn: 0.3417056	test: 0.5030068	best: 0.5021562 (2715)	total: 39m 10s	remaining: 1m 1s
4873:	learn: 0.3416800	test: 0.5030121	best: 0.5021562 (2715)	total: 39m 11s	remaining: 1m
4874:	learn: 0.3416483	test: 0.5030144	best: 0.5021562 (2715)	total: 39m 11s	remaining: 1m
4875:	learn: 0.3416196	test: 0.5030100	best: 0.5021562 (2715)	total: 39m 12s	remaining: 59.8s
4876:	learn: 0.3415831	test: 0.5030054	best: 0.5021562 (2715)	total: 39m 12s	remaining: 59.3s
4877:	learn: 0.3415516	test: 0.5029992	best: 0.5021562 (2715)	total: 39m 12s	remaining: 58.8s
4878:	learn: 0.3415290	test: 0.5030071	best: 0.5021562 (2715)	total: 39m 13s	remaining: 58.4s
4879:	learn: 0.3414885	test: 0.5030093	best: 0.5021562 (2715)	total: 39m 14s	remaining: 57.9s
4880:	learn: 0.3414705	test: 0.5030122	best: 0.5021562 (2715)	total: 39m 14s	remaining: 57.4s
4881:	learn: 0.3414439	test: 0.5030070	best: 0.5021562 (2715)	total

4959:	learn: 0.3395653	test: 0.5030690	best: 0.5021562 (2715)	total: 39m 54s	remaining: 19.3s
4960:	learn: 0.3395306	test: 0.5030703	best: 0.5021562 (2715)	total: 39m 54s	remaining: 18.8s
4961:	learn: 0.3394943	test: 0.5030722	best: 0.5021562 (2715)	total: 39m 55s	remaining: 18.3s
4962:	learn: 0.3394666	test: 0.5030765	best: 0.5021562 (2715)	total: 39m 56s	remaining: 17.9s
4963:	learn: 0.3394423	test: 0.5030779	best: 0.5021562 (2715)	total: 39m 56s	remaining: 17.4s
4964:	learn: 0.3394334	test: 0.5030786	best: 0.5021562 (2715)	total: 39m 57s	remaining: 16.9s
4965:	learn: 0.3394037	test: 0.5030811	best: 0.5021562 (2715)	total: 39m 57s	remaining: 16.4s
4966:	learn: 0.3393832	test: 0.5030828	best: 0.5021562 (2715)	total: 39m 57s	remaining: 15.9s
4967:	learn: 0.3393592	test: 0.5030858	best: 0.5021562 (2715)	total: 39m 58s	remaining: 15.4s
4968:	learn: 0.3393306	test: 0.5030857	best: 0.5021562 (2715)	total: 39m 59s	remaining: 15s
4969:	learn: 0.3393166	test: 0.5030845	best: 0.5021562 (2715)	

A Jupyter Widget

0:	learn: 2.7726935	test: 2.7666900	best: 2.7666900 (0)	total: 446ms	remaining: 37m 7s


In [ ]:
pred_tr2, pred_te2 = model_run('train2')
pred_tr2.to_csv(path+'/'+ver+'_pred_tr2.csv', index=False)
pred_te2.to_csv(path+'/'+ver+'_pred_te2.csv', index=False)

In [ ]:
sub = pd.read_csv('C:/Users/Administrator/Documents/sample_submission.csv')
sub['air_store_id'] = sub.id.map(lambda x: '_'.join(x.split('_')[:-1]))
sub['visit_date']   = sub.id.map(lambda x: x.split('_')[2])
sub['visit_date'] = pd.to_datetime(sub['visit_date'])

pred_te = pd.concat([pred_te1,pred_te2])
pred_te['visit_date'] = pd.to_datetime(pred_te['visit_date'])

# submission
sub.loc[:,'visitors'] = sub.merge(pred_te, on=['air_store_id','visit_date'], how='left')['pred'].values
sub['visitors'] = np.expm1(sub['visitors'])
sub_file = path+'/'+ver+'_sub.csv'
sub[['id','visitors']].to_csv(sub_file, index=False)

# adjusted subumission
sub['pred'] = sub['visitors']
sub.loc[sub.visit_date==date(2017,4,25),'visitors']*=1.10 # Tuesday-judgmental

sub.loc[sub.visit_date==date(2017,4,27),'visitors']*=1.10 # Thursday-judgmental

sub.loc[sub.visit_date==date(2017,4,30),'visitors']*=1.35 # Sunday-judgmental-
sub.loc[sub.visit_date==date(2017,5,1),'visitors'] *=1.20 # Monday-judgmental-should be high because in-between
sub.loc[sub.visit_date==date(2017,5,2),'visitors'] *=1.10 # Tuesday-judgmental-should be high because in-between
sub.loc[sub.visit_date==date(2017,5,3),'visitors'] *=1.10 # Day1
sub.loc[sub.visit_date==date(2017,5,4),'visitors'] *=1.10 # Day2
sub.loc[sub.visit_date==date(2017,5,5),'visitors'] *=1.05 # Day3
sub.loc[sub.visit_date==date(2017,5,6),'visitors'] *=1.00 # Saturday
sub.loc[sub.visit_date==date(2017,5,7),'visitors'] *=0.95 # Sunday
sub.loc[sub.visit_date==date(2017,5,8),'visitors'] *=0.90 # Monday
sub.loc[sub.visit_date==date(2017,5,9),'visitors'] *=0.90 # Tuesday
sub_file = path+'/'+ver+'_sub_adjusted.csv'
sub[['id','visitors']].to_csv(sub_file, index=False)

In [ ]:
# 2016 Act vs 2017 Pred
pred = pd.DataFrame(sub.groupby(['visit_date'])['pred','visitors'].mean().reset_index()).rename(columns={'visitors':'pred_adj'})

train = pd.read_csv('C:/Users/Kohei/Documents/Kaggle/Recruit/00_input/air_visit_data.csv')[['visit_date','visitors']]
train['visit_date'] = pd.to_datetime(train['visit_date'])
train['visit_date'] = train['visit_date']+timedelta(days=365-1)
df = train[train.visit_date>=date(2017,4,23)]
df = df[df.visit_date<=date(2017,5,31)]
df = pd.DataFrame(df.groupby(['visit_date'])['visitors'].mean().reset_index())

df = df.merge(pred, on='visit_date', how='left')

fig = plt.figure(figsize=(15,5))
plt.plot(df.visit_date, df.visitors, "blue", label = "2016")
plt.plot(df.visit_date, df.pred, "orange", label = "Test-original")
plt.plot(df.visit_date, df.pred_adj, "red", label = "Test-adjusted", linestyle='dotted')
plt.ylim(10,30)
plt.legend(loc='upper left')
plt.grid()